This lab on Subset Selection is a Python adaptation of p. 244-247 of "Introduction to Statistical Learning with Applications in R" by Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani. It is based on the work of R. Jordan Crouser at Smith College for SDS293: Machine Learning (Spring 2016).

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import itertools
import time
import statsmodels.api as sm
import statsmodels.datasets as smd
import matplotlib.pyplot as plt

# 6.5.1 Best Subset Selection

Here we apply the best subset selection approach to the Hitters data. We
wish to predict a baseball player’s Salary on the basis of various statistics
associated with performance in the previous year. Let's take a quick look:

In [ ]:
use_smith_edu = False  or True
if use_smith_edu:
    url = ''
    hitters_df = pd.read_csv('https://www.science.smith.edu/~jcrouser/SDS293/data/Hitters.csv')
    hitters_df = hitters_df.set_index('Player')
else:
    hitters_df = smd.get_rdataset('Hitters', 'ISLR')
    hitters_df.data.__doc__ = hitters_df.__doc__
    hitters_df = hitters_df.data
hitters_df.head()

First of all, we note that the `Salary` variable is missing for some of the
players. The `isnull()` function can be used to identify the missing observations. It returns a vector of the same length as the input vector, with a `TRUE` value
for any elements that are missing, and a `FALSE` value for non-missing elements.
The `sum()` function can then be used to count all of the missing elements:

In [ ]:
print("Number of null values:", hitters_df["Salary"].isnull().sum())

We see that `Salary` is missing for 59 players. The `dropna()` function
removes all of the rows that have missing values in any variable:

In [ ]:
# Print the dimensions of the original Hitters data (322 rows x 20 columns)
print("Dimensions of original data:", hitters_df.shape)

# Drop any rows the contain missing values, along with the player names
hitters_df_clean = hitters_df.dropna()

# Print the dimensions of the modified Hitters data (263 rows x 20 columns)
print("Dimensions of modified data:", hitters_df_clean.shape)

# One last check: should return 0
print("Number of null values:", hitters_df_clean["Salary"].isnull().sum())

In [ ]:
dummies = pd.get_dummies(hitters_df_clean[['League', 'Division', 'NewLeague']])

y = hitters_df_clean.Salary

# Drop the column with the independent variable (Salary), and columns for which we created dummy variables
hitters = hitters_df_clean.drop(['League', 'Division', 'NewLeague'], axis=1).astype('float64')

# Define the feature set X.
hitters = pd.concat([hitters, dummies[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)

We can perform best subset selection by identifying the best model that contains a given number of predictors, where **best** is quantified using RSS. We'll define a helper function to outputs the best set of variables for
each model size:

In [ ]:
itertools.combinations?

In [ ]:
from tqdm.auto import tqdm
def haty_norm2(y, X):
    q, r = np.linalg.qr(X)
    return (np.dot(q.T, y)**2).sum()

def best_k(y, X, k):
    best_v = 0
    best_c = None
    for c in itertools.combinations(range(X.shape[1]),k):
        v = haty_norm2(y, X[:,c])
        if v > best_v:
            best_v = v
            best_c = c
    return best_c, best_v

def best_subset(y, X, max_k=8):
    col_names = X.columns
    y, X = np.asarray(y), np.asarray(X)
    y = y - y.mean(axis=0)
    X = X - X.mean(axis=0)
    q,r = np.linalg.qr(X)
    TSS = (y**2).sum()
    y = np.dot(q.T, y)
    def result(x):
        c,v = x
        return {
            'num_pred': len(c),
            'vars': [col_names[i] for i in c],
            'R2': v/TSS
        }
    return pd.DataFrame([result(best_k(y, r, k+1)) for k in tqdm(range(max_k))]).set_index('num_pred')
    

In [ ]:
%%time
fit = pd.DataFrame(best_subset(hitters.Salary, hitters.drop(columns=['Salary']), 19))

In [ ]:
fit

In [ ]:
fit['model'] = [sm.OLS.from_formula(f, data=hitters).fit()
              for f in fit.vars.apply(lambda x: 'Salary ~ ' + ' + '.join(sorted(x)))]

In [ ]:
def traverse_tree(y, X, best_v, best_subset, k=0, v_offset=0):
    idx = np.arange(X.shape[1])
    for i in range(X.shape[1]):
        x, X0 = X[:,i], X[:,idx!=i]
        traverse_tree(y, X0, best_v, best_subset, k=k, v_offset=v_offset)
        x = x/np.linalg.norm(x) 
        X0 = X0 - np.dot(x,X0)
        c = np.dot(x, y)
        y0 = y - c*x
        traverse_tree(y0, X0, best_v, best_subset, k=k+1, v_offset=v_offset+c**2)
        
def bestsubset(y, X):
    y, X = np.asarray(y), np.asarray(X)
    y = y - y.mean(axis=0)
    X = X - X.mean(axis=0)
    q, r = np.linalg.qr(X)
    TSS = (y**2).sum()
    y, X = np.dot(q.T, y), r
    p, = y.shape
    best_v = np.zeros(p)
    best_subset = [None]*p
    traverse_tree()
    return best_v/TSS, best_subset

In [ ]:
# def processSubset(feature_set):
#     # Fit model on feature_set and calculate RSS
#     model = sm.OLS(y,X[list(feature_set)])
#     regr = model.fit()
#     RSS = ((regr.predict(X[list(feature_set)]) - y) ** 2).sum()
#     return {"model":regr, "RSS":RSS}

In [ ]:
# def getBest(k):
    
#     tic = time.time()
    
#     results = []
    
#     for combo in itertools.combinations(X.columns, k):
#         results.append(processSubset(combo))
    
#     # Wrap everything up in a nice dataframe
#     models = pd.DataFrame(results)
    
#     # Choose the model with the highest RSS
#     best_model = models.loc[models['RSS'].argmin()]
    
#     toc = time.time()
#     print("Processed", models.shape[0], "models on", k, "predictors in", (toc-tic), "seconds.")
    
#     # Return the best model, along with some other useful information about the model
#     return best_model

This returns a `DataFrame` containing the best model that we generated, along with some extra information about the model. Now we want to call that function for each number of predictors $k$:

In [ ]:
# # Could take quite a while to complete...

# models_best = pd.DataFrame(columns=["RSS", "model"])

# tic = time.time()
# for i in range(1,4):
#     models_best.loc[i] = getBest(i)

# toc = time.time()
# print("Total elapsed time:", (toc-tic), "seconds.")

Now we have one big `DataFrame` that contains the best models we've generated along with their RSS:

In [ ]:
# models_best['model'][2].summary()

If we want to access the details of each model, no problem! We can get a full rundown of a single model using the `summary()` function:

In [ ]:
print(fit.loc[2, "model"].summary())

This output indicates that the best two-variable model
contains only `Hits` and `CRBI`. To save time, we only generated results
up to the best 7-variable model. You can use the functions we defined above to explore as many variables as are desired.

In [ ]:
# Show the best 19-variable model (there's actually only one)
print(fit.model.iloc()[-1].summary())

Rather than letting the results of our call to the `summary()` function print to the screen, we can access just the parts we need using the model's attributes. For example, if we want the $R^2$ value:

In [ ]:
fit.loc[2, "model"].rsquared

Excellent! In addition to the verbose output we get when we print the summary to the screen, fitting the `OLM` also produced many other useful statistics such as adjusted $R^2$, AIC, and BIC. We can examine these to try to select the best overall model. Let's start by looking at $R^2$ across all our models:

In [ ]:
# Gets the second element from each row ('model') and pulls out its rsquared attribute
print('R2:', fit.model.apply(lambda x: x.rsquared), sep='\n')
print('RSS:', fit.model.apply(lambda x: x.ssr), sep='\n')

As expected, the $R^2$ statistic increases monotonically as more
variables are included.

Plotting RSS, adjusted $R^2$, AIC, and BIC for all of the models at once will
help us decide which model to select.

In [ ]:
# plt.rcParams

In [ ]:
plt.figure(figsize=(20,10))
plt.rcParams.update({'font.size': 16, 'lines.markersize': 8})

# We will now plot a red dot to indicate the model with the largest adjusted R^2 statistic.
# The argmax() function can be used to identify the location of the maximum point of a vector
plt_data = np.stack(fit.apply(lambda x: (len(x['vars']), x['model'].ssr), axis=1))

# Set up a 2x2 grid so we can look at 4 plots at once
plt.subplot(2, 2, 1)
plt.plot(*plt_data.T)
plt.xlabel('# Predictors')
plt.ylabel('RSS')
plt.grid()
plt.plot(*plt_data[plt_data[:,1].argmin()], 'or')

# We will now plot a red dot to indicate the model with the largest adjusted R^2 statistic.
# The argmax() function can be used to identify the location of the maximum point of a vector

plt_data = np.stack(fit.apply(lambda x: (len(x['vars']), x['model'].rsquared_adj), axis=1))

plt.subplot(2, 2, 2)
plt.plot(*plt_data.T)
plt.xlabel('# Predictors')
plt.ylabel('adjusted rsquared')
plt.grid()
plt.plot(*plt_data[plt_data[:,1].argmax()], 'or')


# We'll do the same for AIC and BIC, this time looking for the models with the SMALLEST statistic
plt_data = np.stack(fit.apply(lambda x: (len(x['vars']), x['model'].aic), axis=1))

plt.subplot(2, 2, 3)
plt.plot(*plt_data.T)
plt.xlabel('# Predictors')
plt.ylabel('AIC')
plt.grid()
plt.plot(*plt_data[plt_data[:,1].argmin()], 'or')


plt_data = np.stack(fit.apply(lambda x: (len(x['vars']), x['model'].bic), axis=1))

plt.subplot(2, 2, 4)
plt.plot(*plt_data.T)
plt.xlabel('# Predictors')
plt.ylabel('BIC')
plt.grid()
plt.plot(*plt_data[plt_data[:,1].argmin()], 'or')


Recall that in the second step of our selection process, we narrowed the field down to just one model on any $k<=p$ predictors. We see that according to BIC, the best performer is the model with 6 variables. According to AIC and adjusted $R^2$ something a bit more complex might be better. Again, no one measure is going to give us an entirely accurate picture... but they all agree that a model with 5 or fewer predictors is insufficient.

# 6.5.2 Forward and Backward Stepwise Selection
We can also use a similar approach to perform forward stepwise
or backward stepwise selection, using a slight modification of the functions we defined above:

In [ ]:
def forward(y, X):
    col_names = list(X.columns)
    y, X = np.asarray(y), np.asarray(X)
    y = y - y.mean(axis=0)
    X = X - X.mean(axis=0)
    q, X = np.linalg.qr(X)
    TSS = (y**2).sum()
    y = np.dot(q.T, y)
    tv = 0
    result = []
    predictors = []
    while len(col_names)>0:
        (c,), v = best_k(y, X, 1)
        tv += v
        predictors.append(col_names.pop(c))
        result.append(dict(num_pred=len(predictors), predictors=predictors.copy(), r2=tv/TSS))
        x, X = X[:,c], X[:,np.arange(X.shape[1])!=c] 
        x = x/np.linalg.norm(x)
        y = y - np.dot(y,x)*x
        X = X - np.outer(x, np.dot(x.T, X))
    return pd.DataFrame(result).set_index('num_pred')


Now let's see how much faster it runs!

In [ ]:
fwd_fit = forward(hitters.Salary, hitters.drop(columns=['Salary']))
fwd_fit

In [ ]:
def regress_on(v):
    return sm.OLS.from_formula('Salary ~ ' + ' + '.join(sorted(v)), data=hitters).fit()

fwd_fit['model'] = [regress_on(v) for v in fwd_fit.predictors]

Phew! That's a lot better. Let's take a look:

In [ ]:
fwd_fit.apply(lambda x: (x['model'].rsquared, x['r2']), axis=1)

In [ ]:
print(fwd_fit.loc[1, "model"].summary())
print(fwd_fit.loc[2, "model"].summary())

We see that using forward stepwise selection, the best one-variable
model contains only `Hits`, and the best two-variable model additionally
includes `CRBI`. Let's see how the models stack up against best subset selection:

In [ ]:
print(fit.loc[7, "model"].params)
print(fwd_fit.loc[7, "model"].params)

For this data, the best one-variable through six-variable
models are each identical for best subset and forward selection.

# Backward Selection
Not much has to change to implement backward selection... just looping through the predictors in reverse!

In [ ]:
def backward(y, X):
    col_names = list(X.columns)
    y, X = np.asarray(y), np.asarray(X)
    y = y - y.mean(axis=0)
    X = X - X.mean(axis=0)
    q, X = np.linalg.qr(X)
    TSS = (y**2).sum()
    y = np.dot(q.T, y)
    tv = 0
    result = []
    while len(col_names)>1:
        c, v = best_k(y, X, len(col_names)-1)
        col_names = [col_names[i] for i in c]
        result.append(dict(num_pred=len(c), 
                           predictors=col_names, 
                           r2=v/TSS))
        X = X[:,c] 
    return pd.DataFrame(result).set_index('num_pred')


In [ ]:
bwd_fit = backward(hitters.Salary, hitters.drop(columns=['Salary']))
print(bwd_fit)
bwd_fit['model'] = [regress_on(v) for v in bwd_fit.predictors]

For this data, the best one-variable through six-variable
models are each identical for best subset and forward selection.
However, the best seven-variable models identified by forward stepwise selection,
backward stepwise selection, and best subset selection are different:

In [ ]:
print("------------")
print("Best Subset:")
print("------------")
print(fit.loc[7, "model"].params)

In [ ]:
print("-----------------")
print("Foward Selection:")
print("-----------------")
print(fwd_fit.loc[7, "model"].params)

In [ ]:
print("-------------------")
print("Backward Selection:")
print("-------------------")
print(bwd_fit.loc[7, "model"].params)